In [ ]:
from copy import deepcopy
import json
import os
from typing import List
import uuid
import re
import pandas as pd

## Utilidades

In [ ]:
def drop_duplicates(ls: list) -> list:
    return list(filter(None, set(ls)))

def flatten(ls: list) -> list:
    return [item for sublist in ls for item in sublist]

def remove(ls: list, to_remove: list) -> list:
    return [item for item in ls if item not in to_remove]

def plural(palavra: str) -> str:
    """
    Passa uma palavra em português para o plural. Não funciona sempre,
    devido à quantidade de exceções que precisam ser programadas à mão.
    """
    palavra = palavra.strip()
    invariaveis = [r"x$"]
    for p in invariaveis:
        if re.search(p, palavra):
            return palavra
    substituicoes = {
        r"ão": r"õe",
        r"r$": r"re",
        r"z$": r"ze",
        r"s$": r"se",
        r"(?<=[aeou])l": r"i",
        r"il": r"ei",
        r"m$": r"n",
    }
    for p, s in substituicoes.items():
        palavra = re.sub(p, s, palavra)
    return palavra + "s" if palavra[-1] != "s" else palavra

def sanitize(palavra: str) -> str:
    """Substitui caracteres acentuados."""
    substituicoes = {
        "a": ["á", "â", "ã", "à"],
        "c": ["ç"],
        "e": ["é", "ê"],
        "i": ["í"],
        "o": ["ó", "ô", "õ"],
        "u": ["ú", "ü"],
    }
    for substituta, letras in substituicoes.items():
        for letra in letras:
            palavra = palavra.replace(letra, substituta)
    return palavra

In [ ]:
def is_flat(x):
    return not (isinstance(x, list) or isinstance(x, dict))

def is_flat_list(x):
    return all(is_flat(y) for y in x)

def mix_list(a, b):
    """
    Realiza um outer join entre duas listas planas ou que contêm dicionários.
    """
    if is_flat_list(a) and is_flat_list(b):
        return drop_duplicates(a + b)
    elif is_flat_list(a) or is_flat_list(b):
        raise ValueError(a, b)
    
    a_dict = to_dict(a)
    b_dict = to_dict(b)
    mixed_dicts = mix_dict(a_dict, b_dict)
    return list(mixed_dicts.values())

def to_dict(ls: List[dict]):
    """
    Transforma uma lista de dicionários em um dicionário de dicionários, 
    inferindo como chave para cada dict algum valor dele que seja presente
    em todos os dicts, único em cada um e plano.
    """
    if len(ls) == 1:
        return {"placeholder_key": ls[0]}
    d = ls[0]
    global_key = None
    for key in d:
        # se a chave for uma das hardcoded
        if key in ["conditions"]:
            global_key = key
            break
        # se o valor dessa chave não for plano
        if not is_flat(d[key]):
            continue
        # se nem todos dicts tiverem essa chave
        if not all(key in d_ for d_ in ls):
            continue
        # se nem todos os dicts tiverem um valor para essa chave
        if not all(bool(d_[key]) for d_ in ls):
            continue
        # se cada dict não tiver um valor único para essa chave
        if not len(set(d_[key] for d_ in ls)) == len(ls):
            continue
        # se já tiver encontrado uma chave global
        if global_key:
            raise ValueError(f"Mais de uma possível chave global encontrada para {ls}: {key}, {global_key}")
        global_key = key
        
    if not global_key:
        raise ValueError(f"Não encontrei chave global para {ls}")
    return {d_[global_key]: d_ for d_ in ls}
    
def mix_dict(a, b):
    """
    Retorna um dicionário contendo as chaves e valores de ambos dicionários
    de entrada, dando preferência para os valores de b.
    """
    out = deepcopy(b)
    for k, v in a.items():
        if k not in out:
            out[k] = v
        elif isinstance(v, list):
            if not isinstance(b[k], list):
                raise ValueError(f"a é uma lista em {k}, mas b é {type(b[k])}")
            out[k] = mix_list(v, b[k])
        elif isinstance(v, dict):
            if not isinstance(b[k], dict):
                raise ValueError(f"a é um dict em {k}, mas b é {type(b[k])}")
            out[k] = mix_dict(v, b[k])
        else:
            out[k] = v
    return out
            

## I/O

In [ ]:
def load_perguntas():
    df = pd.read_excel("results/Perguntas.xlsx", sheet_name="finais")
    df = df.dropna(subset=["Resposta"])
    subs = {
        "Pergunta": "pergunta",
        "Resposta": "resposta",
        "Intenção": "intent",
        "Rótulos": "rótulos",
        "Modificador": "modificador",
        "Substantivo": "substantivo",
        "Recipiente": "recipiente",
        "Elocuções": "examples",
    }
    df = df[list(subs.keys())]
    df = df.rename(columns=subs)
    df = df.fillna("")
    subset_accents = ["rótulos", "modificador", "substantivo", "recipiente"]
    return df

def load_skill(skill_path):
    with open(skill_path, "r", encoding="utf-8") as f:
        skill = json.load(f)
    return skill

def save_skill(skill_path, skill):
    root, extension = os.path.splitext(skill_path)
    new_skill_path = f"{root}2{extension}"
    with open(new_skill_path, "w", encoding="utf-8") as f:
        json.dump(skill, f, ensure_ascii=False)

## Intenções

In [ ]:
def get_intents(df: pd.DataFrame):
    subset = ["intent", "pergunta", "examples"]
    records = df[subset].to_dict(orient="records")
    intents = [
        {
            "intent": record["intent"],
            "examples": get_examples(record),
            "description": "",
        }
        for record in records
    ]
    return intents

def get_examples(record: dict) -> List:
    # a própria pergunta é um exemplo
    out = [{"text": record["pergunta"]}]
    # tudo que está em Elocuções é exemplo também
    if record["examples"]:
        out += [{"text": exemplo} for exemplo in record["examples"].split("--")]
    return out

## Entidades

In [ ]:
def get_entity_values(series):
    records = series.drop_duplicates().to_list()
    records = [r.split("-") for r in records]
    records = flatten(records)
    records = drop_duplicates(records)
    values = [
        {"type": "synonyms", "value": record, "synonyms": []} for record in records
    ]
    return values

def get_entities(df):
    subset = ["rótulos", "modificador", "substantivo", "recipiente"]
    entities = [
        {"entity": col, "values": get_entity_values(df[col]), "fuzzy_match": True}
        for col in subset
    ]
    return entities

## Nós de diálogo

In [ ]:
ls = ["listar", "crustáceo"]
ls[0] = ls[0].capitalize()
ls

In [ ]:
def get_dialog_nodes(df):
    records = df.to_dict(orient="records")
    dialog_nodes = [
        {
            "type": "standard",
            "title": get_titulo(record),
            "output": {
                "generic": [
                    {
                        "values": [{"text": record["resposta"]}],
                        "response_type": "text",
                        "selection_policy": "sequential",
                    }
                ]
            },
            "context": {"contexto": record["rótulos"]},
            "conditions": get_condition_string(record),
            "dialog_node": f"node_{uuid.uuid4().hex[:16]}",
        }
        for record in records
    ]
    return dialog_nodes

def get_titulo(js: dict) -> str:
    modificador = js["modificador"]
    substantivo = js["substantivo"].replace("-", " ")
    recipiente = js["recipiente"]
    
    contextos = get_contextos(js["rótulos"].split("-"))
    contextos = list(filter(lambda x: x.count("-") == 0, contextos))
    trechos = []
    if contextos:
        trechos.append(f"{'/'.join(contextos)}:")
    
    if modificador in ["efeito"]:
        trechos.append(modificador)
        if substantivo:
            trechos.append("de {substantivo}")
        recipiente = recipiente or contextos[0]
        trechos.append(f"em {recipiente}")
    elif modificador in ["maiores", "menores"]:
        substituir = {"produção": "produtores"}
        if recipiente in substituir:
            recipiente = substituir[recipiente]
        substantivo = plural(substantivo)
        trechos.append(f"{modificador} {substantivo} {recipiente}?")
    elif modificador in ["diferença"]:
        recipiente = recipiente or contextos[0]
        trechos.append(f"{modificador} entre {substantivo} e {recipiente}?")
    elif modificador in ["existe", "quantidade"]:
        recipiente = recipiente or "no Brasil"
        trechos.append(modificador)
        if substantivo:
            trechos.append(substantivo + " ")
        trechos.append(f"{recipiente}?")
    elif modificador in ["listar"]:
        substituir = {"extinção": "em extinção", "aaz": "na Amazônia Azul"}
        if recipiente in substituir:
            recipiente = substituir[recipiente]
        substantivo = plural(substantivo)
        trechos.append(f"listar {substantivo} {recipiente}")
    elif modificador in ["pertence"]:
        if substantivo and recipiente:
            raise ValueError("Perguntas do tipo 'pertence' precisam de substantivo ou de recipiente!")
        elif substantivo:
            trechos.append(f"{substantivo} é um {contextos[0]}?")
        elif recipiente:
            trechos.append(f"{contextos[0]} é um {recipiente}?")
        else:
            raise ValueError("Perguntas do tipo 'pertence' precisam de substantivo ou de recipiente!")
    else:
        trechos.append(modificador)
        if substantivo:
            trechos.append(substantivo)
        if recipiente:
            trechos.append(recipiente)
    trechos[0] = trechos[0].capitalize()
    titulo = " ".join(trechos)
    titulo = titulo.strip()
    return titulo

def get_contextos(rotulos):
    """
    Devolve os contextos de uma pergunta baseado em seus rótulos,
    sendo que há uma lista de rótulos que não definem contexto.
    """
    rotulos_nao_contextuais = [
        "fauna",
        "flora",
        "extra",
        "física",
        "símbolo",
        "turismo",
        "engenharia",
        "saúde",
        "geologia",
    ]
    contextos = [contexto for contexto in rotulos if all(rot not in contexto for rot in rotulos_nao_contextuais)]
    return contextos

def get_condition_string(js):
    modificador = js["modificador"]
    substantivo = js["substantivo"]
    recipiente = js["recipiente"]
    rotulos = js["rótulos"].split("-") + [js["rótulos"]]
    rotulos = drop_duplicates(rotulos)
    contextos = get_contextos(rotulos)

    if contextos:
        conds_adicionais = [get_condition(modificador, substantivo, recipiente, contexto) for contexto in contextos]
    else:
        conds_adicionais = [get_condition(modificador, substantivo, recipiente)]

    conds = [f"#{js['intent']}"] + flatten(conds_adicionais)
    cond_str = " || ".join(conds)
    cond_str = sanitize(cond_str)
    return cond_str

def get_condition(modificador, substantivo, recipiente, contexto=None) -> list:
    trechos = [f"@modificador:{modificador}"]
    if substantivo:
        trechos.append(f"&& @substantivo:{substantivo}")
    if recipiente:
        trechos.append(f"&& @recipiente:{recipiente}")
    cond = " ".join(trechos)
    if contexto:
        out = [cond + f' && $contexto=="{contexto}"', cond + f" && @rótulos:{contexto}"]
    else:
        out = [cond]
    return out
                    
def apply_previous_siblings(dialog_nodes: dict) -> dict:
    # atribui previous siblings
    for i in range(len(dialog_nodes) - 1):
        prev, node = dialog_nodes[i], dialog_nodes[i + 1]
        node["previous_sibling"] = prev["dialog_node"]
    return dialog_nodes

## Skill

In [ ]:
def mix_skills(skill, **kwargs):
    new_skill = deepcopy(skill)
    for k, v in kwargs.items():
        new_skill[k] = v
    return new_skill

## Rodar

In [ ]:
df = load_perguntas()
df[~(df["examples"]=="")]

In [ ]:
skill_path = "results/skill-Amazônia-Azul.json"
skill = load_skill(skill_path)

In [ ]:
new_intents = mix_list(skill["intents"], get_intents(df))
new_intents

In [ ]:
new_entities = mix_list(skill["entities"], get_entities(df))
new_entities

In [ ]:
new_dialog_nodes = mix_list(skill["dialog_nodes"], get_dialog_nodes(df))
new_dialog_nodes = apply_previous_siblings(new_dialog_nodes)
new_dialog_nodes

In [ ]:
new_skill = mix_skills(skill, intents=new_intents, entities=new_entities, dialog_nodes=new_dialog_nodes)
save_skill(skill_path, new_skill)

## Comparar

In [ ]:
nodes_dict = to_dict(nodes)
new_nodes_dict = to_dict(new_dialog_nodes)
diff = [node for node in new_nodes_dict if node not in nodes_dict]
diff = [new_nodes_dict[node] for node in diff]
diff